# Subsections Issue

## Setup

### Imports

In [1]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Logging

### Initialize 2-line Section Tests

#### 2-line Section Source

In [2]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

**********

### Initial Section and Sub-Section Definitions

- Only definition line is:<br>
`subsections=[sub_section]`

In [3]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Have 1 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from future_items
Buffered Iterator    - DEBUG: Have 1 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: More text to be ignored	 from source


[['Text to be ignored',
  'StartSection Name: A',
  'EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in all lines in one sub-list

**********

## Section Break options

### Add start to Section Definition

- Section start **Before** *StartSection*

`start_section=SectionBreak('StartSection', break_offset='Before'),`

In [4]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: More text to be ignored	 from source


[['StartSection Name: A',
  'EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Includes all lines after first *StartSection* in single sub-list.

### Add end to Section Definition

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [5]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Includes both lines of first section in single sub-list.

### Define Muti-Section to Read Both Sections

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )
```

In [6]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Have 1 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item

[[['StartSection Name: A', 'EndSection Name: A']],
 [['StartSection Name: B', 'EndSection Name: B']],
 [[]]]


- ![Bad](../examples/error.png) Includes both lines of each section in its own sub-list, but adds a blank sub-list at the end.

### Set Same Start and End Breaks for Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *StartSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )
```

In [7]:
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )

multi_section = Section(section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Have 1 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from future_items
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']]


- Without SubSection, it works as expected.

- ![Bad](../examples/error.png) Appears to hang

**********

## SubSection Break Options

### Add *End on First* to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_on_first_item=True,
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [8]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[['EndSection Name: A']]


- ![Bad](../examples/error.png) Results in `['EndSection Name: A']` sub-list, should have been `['StartSection Name: A']`.

### Add *End* as **True** to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection end is **True**  Should be the same as **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_section=SectionBreak(True)
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [9]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[['StartSection Name: A'], []]


## What should  `end_section=SectionBreak(True)` result in?

- ![Bad](../examples/error.png) Results in `['StartSection Name: A'], []` sub-lists, should have been `['StartSection Name: A']`.

### Add *Start* to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [10]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[[]]


- ![Bad](../examples/error.png) Results in blank sub-list.

### Change Start Sub-Section to **After*

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [11]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[[]]


- ![Bad](../examples/error.png) Results in blank sub-list.

# DONE TO HERE

### Add start to SubSection Definition
> `start_section=SectionBreak('EndSection', break_offset='Before')`

In [12]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Have 1 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: More text to be ignored	 from source


[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) Results in Empty list of lists

### Add *Start After* to Section Definition
> `start_section=SectionBreak('StartSection', break_offset='After')`

In [13]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: More text to be ignored	 from source


[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before')`
    

In [14]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: More text to be ignored	 from source


[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/error.png) Results in first and second section

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition and set *End On First Line* for SubSection
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_on_first_item=True,`
    

In [15]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: B	 from source
Buffered Iterator    - DEBUG: Getting item: More text to be ignored	 from source


[['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']]


- ![Bad](../examples/Error.png) Results in second section

### Add *Start After StartSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [16]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 5 Previous Items
Buffered Iterato

[['EndSection Name: A'], ['EndSection Name: B'], []]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start After StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [17]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [18]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[[]]


- ![Bad](../examples/Error.png) Results in empty list of lists

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to *Before EndSection*, *End* to *`True` (Always Break)* and *Keep Partial* to *`True`*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`<br>
> `keep_partial=True,`
    

In [19]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[[]]


- ![Bad](../examples/Error.png) Results in empty list of lists

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [20]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 2 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in Two line section

### Add *Start __After__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [21]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

Buffered Iterator    - DEBUG: Getting item: Text to be ignored	 from source
Buffered Iterator    - DEBUG: Getting item: StartSection Name: A	 from source
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from source
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Have 3 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back
Buffered Iterator    - DEBUG: Getting item: EndSection Name: A	 from future_items
Buffered Iterator    - DEBUG: Getting item: StartSection Name: B	 from source
Buffered Iterator    - DEBUG: Have 4 Previous Items
Buffered Iterator    - DEBUG: Need 1 Steps back


[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in one line section